In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LassoCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold

In [14]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
from sklearn import metrics

import sys, os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import math
import pickle

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import optuna
from optuna.samplers import TPESampler

In [3]:
#https://github.com/dhaitz/mplcyberpunk
import mplcyberpunk

In [4]:
sys.path.append(os.path.realpath('..'))

In [45]:
train_df = pd.read_csv(r"..\data\train.csv", index_col=0)
test_df = pd.read_csv(r"..\data\test.csv", index_col=0)
sample_sub = pd.read_csv(r"..\data\sample_submission.csv", index_col=0)

In [46]:
del train_df['Over18']
del train_df['EmployeeCount']
del train_df['StandardHours']

In [7]:
columns_to_vectorize = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
for vector_target in columns_to_vectorize:
    print(vector_target)
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train_df[vector_target])
    train_df[f'{vector_target}_v'] = vectorizer.transform(train_df[vector_target]).toarray().argmax(axis=1)[:,None]
    vectorizer.fit_transform(test_df[vector_target])
    test_df[f'{vector_target}_v'] = vectorizer.transform(test_df[vector_target]).toarray().argmax(axis=1)[:,None]

BusinessTravel
Department
EducationField
Gender
JobRole
MaritalStatus
OverTime


In [8]:
features = ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction','HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
        'BusinessTravel_v', 'Department_v', 'EducationField_v','Gender_v', 'JobRole_v', 'MaritalStatus_v', 'OverTime_v']
target = ['Attrition']

In [11]:
scaler = MinMaxScaler().fit(train_df[features])
X = scaler.transform(train_df[features])
X_test = scaler.transform(test_df[features])

scaler = MinMaxScaler().fit(train_df[target])
Y = scaler.transform(train_df[target])

In [17]:
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, stratify=Y, test_size=0.4)

In [24]:
len(X_train)

1006

In [40]:
def lassocv_objective(trial):
    # Split the train data for each trial.
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, stratify=Y, test_size=0.2)

    lasso_cv = LassoCV(precompute="auto",
                       fit_intercept=True,
                       max_iter=10000,
                       verbose=False,
                       eps=trial.suggest_float('eps', 0.00001, 0.001),
                       cv=trial.suggest_int('cv', 2, 10),
                       n_alphas=trial.suggest_int('n_alphas', 100, 5000),
                       n_jobs=-1)
    
    lasso_cv.fit(X_train, y_train.ravel())

    preds = lasso_cv.predict(X_train)

    fpr, tpr, _ = roc_curve(y_train, preds)
    roc_auc = auc(fpr, tpr)

    lassocv_ranks[roc_auc] = lasso_cv

    return roc_auc


In [41]:
lassocv_ranks = {}
    
train_time = 1 * 60 * 1
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name='LASSOCVRegressor')
study.optimize(lassocv_objective, timeout=train_time)

[I 2023-01-17 19:03:55,379] A new study created in memory with name: LASSOCVRegressor
[I 2023-01-17 19:03:57,185] Trial 0 finished with value: 0.8183220466516177 and parameters: {'eps': 0.0007195194732056083, 'cv': 5, 'n_alphas': 4004}. Best is trial 0 with value: 0.8183220466516177.
[I 2023-01-17 19:03:57,638] Trial 1 finished with value: 0.8367381489841986 and parameters: {'eps': 0.0009683267919011402, 'cv': 2, 'n_alphas': 2511}. Best is trial 1 with value: 0.8367381489841986.
[I 2023-01-17 19:03:59,846] Trial 2 finished with value: 0.8330511662904441 and parameters: {'eps': 0.0003189565829806228, 'cv': 5, 'n_alphas': 4841}. Best is trial 1 with value: 0.8367381489841986.
[I 2023-01-17 19:04:00,244] Trial 3 finished with value: 0.8368886380737396 and parameters: {'eps': 0.0005944860248105126, 'cv': 7, 'n_alphas': 638}. Best is trial 3 with value: 0.8368886380737396.
[I 2023-01-17 19:04:00,406] Trial 4 finished with value: 0.8308126410835215 and parameters: {'eps': 0.00088497499817211

In [42]:
lassocv_ranks_bytes = pickle.dumps(lassocv_ranks)
with open("rendered_data/lassocv_clsfs_bytes.hex", "wb") as binary_file:
    binary_file.write(lassocv_ranks_bytes)

In [43]:
percent = math.ceil(len(lassocv_ranks.keys()) * .3)
top = sorted(list(lassocv_ranks.keys()))[-percent:]

train_preds = []
test_preds = []
for key in tqdm(top):
    train_preds.append(lassocv_ranks[key].predict(X))
    test_preds.append(lassocv_ranks[key].predict(X_test))

train_final_preds = np.stack(train_preds).mean(0)
test_final_preds = np.stack(test_preds).mean(0)

100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 2499.88it/s]


In [44]:
LASSOCV_train_submission = pd.DataFrame(data={'id': train_df.index, target[0]: train_final_preds})
LASSOCV_train_submission.to_csv(fr'rendered_data/LASSOCV_train_submission.csv', index=False)

LASSOCV_test_submission = pd.DataFrame(data={'id': test_df.index, target[0]: test_final_preds})
LASSOCV_test_submission.to_csv(fr'rendered_data/LASSOCV_test_submission.csv', index=False)